In [1]:
import os

import torch
from torch import nn

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import librosa

import joblib

from typing import List

# Model

In [2]:
class MusicClassifier(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(
                in_features=input_features, out_features=256, dtype=torch.float32
            ),
            nn.GELU(),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=128, dtype=torch.float32),
            nn.GELU(),
            nn.Dropout(p=0.2),
            nn.Linear(
                in_features=128, out_features=output_features, dtype=torch.float32
            ),
        )

    def forward(self, x):
        return self.linear_layer_stack(x)

# Functions

In [3]:
genre_mapping = {
    0: "Blues",
    1: "Classical",
    2: "Country",
    3: "Disco",
    4: "Hiphop",
    5: "Jazz",
    6: "Metal",
    7: "Pop",
    8: "Reggae",
    9: "Rock",
}

In [4]:
genre_mapping_inverse = {
    "Blues": 0,
    "Classical": 1,
    "Country": 2,
    "Disco": 3,
    "Hiphop": 4,
    "Jazz": 5,
    "Metal": 6,
    "Pop": 7,
    "Reggae": 8,
    "Rock": 9,
}

In [5]:
column_names = [
    "chroma_stft_mean",
    "chroma_stft_var",
    "rms_mean",
    "rms_var",
    "spectral_centroid_mean",
    "spectral_centroid_var",
    "spectral_bandwidth_mean",
    "spectral_bandwidth_var",
    "rolloff_mean",
    "rolloff_var",
    "zero_crossing_rate_mean",
    "zero_crossing_rate_var",
    "harmony_mean",
    "harmony_var",
    "tempo",
    "mfcc1_mean",
    "mfcc1_var",
    "mfcc2_mean",
    "mfcc2_var",
    "mfcc3_mean",
    "mfcc3_var",
    "mfcc4_mean",
    "mfcc4_var",
    "mfcc5_mean",
    "mfcc5_var",
    "mfcc6_mean",
    "mfcc6_var",
    "mfcc7_mean",
    "mfcc7_var",
    "mfcc8_mean",
    "mfcc8_var",
    "mfcc9_mean",
    "mfcc9_var",
    "mfcc10_mean",
    "mfcc10_var",
    "mfcc11_mean",
    "mfcc11_var",
    "mfcc12_mean",
    "mfcc12_var",
    "mfcc13_mean",
    "mfcc13_var",
    "mfcc14_mean",
    "mfcc14_var",
    "mfcc15_mean",
    "mfcc15_var",
    "mfcc16_mean",
    "mfcc16_var",
    "mfcc17_mean",
    "mfcc17_var",
    "mfcc18_mean",
    "mfcc18_var",
    "mfcc19_mean",
    "mfcc19_var",
    "mfcc20_mean",
    "mfcc20_var",
]

In [6]:
def get_3sec_sample(song_path) -> list:
    # TODO Vérif
    # TODO Rewrite
    audio, sample_rate = librosa.load(song_path, sr=None)

    segment_duration = 3  # Durée de chaque segment en secondes
    segment_length = int(sample_rate * segment_duration)
    segments = []

    # Effectuez la prédiction toutes les 3 secondes
    for i in range(0, len(audio), segment_length):
        segment = audio[i : i + segment_length]
        segments.append(segment)

    return segments

In [7]:
def audio_pipeline(audio):
    features = []

    # Chromagram
    chroma_stft = librosa.feature.chroma_stft(y=audio)
    features.append(np.mean(chroma_stft))
    features.append(np.var(chroma_stft))  # var => variance

    # RMS (Root Mean Square value for each frame)
    rms = librosa.feature.rms(y=audio)
    features.append(np.mean(rms))
    features.append(np.var(rms))

    # Calcul du Spectral centroid
    spectral_centroids = librosa.feature.spectral_centroid(y=audio)
    features.append(np.mean(spectral_centroids))
    features.append(np.var(spectral_centroids))

    # Spectral bandwith
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio)
    features.append(np.mean(spectral_bandwidth))
    features.append(np.var(spectral_bandwidth))

    # Calcul du spectral rolloff point
    rolloff = librosa.feature.spectral_rolloff(y=audio)
    features.append(np.mean(rolloff))
    features.append(np.var(rolloff))

    # Calcul du ZCR (Zero Crossing Rate)
    zcr = librosa.zero_crossings(audio)
    # features.append(np.sum(zcr))  # Custom
    features.append(np.mean(zcr))
    features.append(np.var(zcr))

    # Harmonic
    harmony = librosa.effects.harmonic(y=audio)
    features.append(np.mean(harmony))
    features.append(np.var(harmony))

    # Tempo
    tempo = librosa.feature.tempo(y=audio)
    features.append(tempo[0])

    # Calcul des moyennes des MFCC
    mfcc = librosa.feature.mfcc(y=audio)
    for x in mfcc:
        features.append(np.mean(x))
        features.append(np.var(x))

    return features

In [8]:
def preprocess_data(song_path):
    scaler = joblib.load("../resources/standard_scaler_pytorch_model_last.pkl")
    dfs = []
    segments = get_3sec_sample(song_path)

    for audio in segments:
        # Perform audio feature extraction
        features = audio_pipeline(audio)

        # Scale the features using the loaded scaler
        scaled_features = scaler.transform([features])

        # Create a DataFrame
        df = pd.DataFrame(scaled_features, columns=column_names)
        dfs.append(df)
    
    return dfs

    # print(len(dfs), " 3sec samples extracted and preprocessed")


In [9]:
def concat_dfs(dfs: List[pd.DataFrame], real_class):
    # Creating new df from processed input
    for i in range(len(dfs)):
        if i == 0:
            new_df = pd.DataFrame(dfs[i], columns=column_names)
        else:
            new_df = pd.concat([new_df, dfs[i]], axis=0)
    new_df["label"] = real_class

    # Créer dataset enrichie (csv)
    original_df = pd.read_csv(
        "./csv/actual_dataset.csv"
    )

    concatened_dataset = pd.concat([original_df, new_df], axis=0)

    concatened_dataset.to_csv("./csv/actual_dataset.csv", index=False)

In [10]:
def training_loop(MusicClassifier):
    # TODO: Externalise this
    # Init le model
    torch.manual_seed(42)
    model = MusicClassifier(input_features=55, output_features=10)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.011)

    def accuracy_fn(y_true, y_pred):
        correct = (
            torch.eq(input=y_true, other=y_pred).sum().item()
        )  # torch.eq() calculates where two tensors are equal
        acc = (correct / len(y_pred)) * 100  # Calcul simple de pourcentage
        return acc

    # Prepare data
    df = pd.read_csv("./csv/actual_dataset.csv")
    # df = pd.read_csv("/app/resources/original_dataset.csv")
    X = torch.from_numpy(df.drop(columns=["label"]).to_numpy()).type(torch.float32)
    y = torch.from_numpy(df["label"].to_numpy()).type(torch.long)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Training loop
    torch.manual_seed(42)
    epochs = 125
    for epoch in range(epochs + 1):
        """
        Train
        """
        model.train()

        # 1. Forward pass
        y_logits = model(X_train)
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

        # 2. Metrics
        loss = loss_fn(y_logits, y_train)
        acc = accuracy_fn(y_true=y_train, y_pred=y_pred)

        # 2.1 Save metrics
        # loss_history.append(loss.cpu().detach().numpy())
        # acc_history.append(acc)

        # 3. Zero Grad
        optimizer.zero_grad()

        # 4. Backpropagation
        loss.backward()

        # 5. Optimmizer step
        optimizer.step()

        """
        Test
        """
        model.eval()

        with torch.inference_mode():
            # 1. Forward pass
            y_test_logits = model(X_test)
            y_test_pred = torch.softmax(y_test_logits, dim=1).argmax(dim=1)

            # 2. Metrics
            test_loss = loss_fn(y_test_logits, y_test)
            test_acc = accuracy_fn(y_pred=y_test_pred, y_true=y_test)

            # 2.1 Save metrics
            # test_loss_history.append(test_loss.cpu().detach().numpy())
            # test_acc_history.append(test_acc)

        # Print out what's happening
        if epoch % 25 == 0:
                print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

    # Save the model
    torch.save(obj=model.state_dict(), f="./actual_model_fast.pth")

# Main

In [11]:
# TODO: Print steps

for folder_class in os.listdir('./extra_music/'):
    for song in os.listdir(f'./extra_music/{folder_class}'):

        # Preprocess input data
        print("1. Preprocessing")
        dfs = preprocess_data(f'./extra_music/{folder_class}/{song}')

        # Load the weights
        print("2.0 Loading the model")
        my_model = MusicClassifier(input_features=55, output_features=10)
        my_model.load_state_dict(
            torch.load(
                f="./actual_model_fast.pth", map_location=torch.device("cpu")
            )
        )

        # Predict 
        my_model.eval()

        class_predictions = []
        raw_results = []

        for df in dfs:
            y_logits = my_model(torch.from_numpy(df.to_numpy()).type(torch.float32))
            y_softmax = torch.softmax(y_logits, dim=1)
            y_pred = y_softmax.argmax(dim=1)

            # print(genre_mapping[y_pred.detach().numpy()[0]])
            # print(list(torch.round(y_softmax * 1000) / 1000))

            raw_results.append(y_softmax.detach().numpy())
            class_predictions.append(genre_mapping[y_pred.detach().numpy()[0]])

        unique_values = set(class_predictions)
        actual_best = 0
        for elt in unique_values:
            if class_predictions.count(elt) > actual_best:
                actual_best = class_predictions.count(elt)
                prediction = elt
            print(elt, class_predictions.count(elt))

        print("Results =>", prediction, "(", genre_mapping_inverse[prediction], ")")

        # Filter kept data
        real_class = int(folder_class)
        print("real_class", real_class)
        kept_dfs: List[pd.DataFrame] = []

        for i in range(len(dfs)):
            sorted_array = np.sort(raw_results[i][0])[::-1]
            # Keep only dfs when the real class i the first or second predicted by the model !
            if raw_results[i][0][real_class] == sorted_array[0] or (raw_results[i][0][real_class] == sorted_array[1] and raw_results[i][0][real_class] > 0):
                # print(raw_results[i][0][real_class])
                kept_dfs.append(dfs[i])

        # print("df_length =>",len(kept_dfs))

        concat_dfs(dfs, real_class)

        training_loop(MusicClassifier)

1. Preprocessing


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
/home/payetquentin/Documents/workspace/briefs/S9_music/Music/.venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/home/payetquentin/Documents/workspace/briefs/S9_music/Music/.venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


2.0 Loading the model
Jazz 6
Rock 9
Blues 4
Hiphop 34
Pop 1
Classical 6
Reggae 26
Disco 2
Results => Hiphop ( 4 )
real_class 2
Epoch: 0 | Loss: 2.28855, Acc: 13.12% | Test Loss: 1.87892, Test Acc: 37.05%
Epoch: 25 | Loss: 0.60456, Acc: 78.64% | Test Loss: 0.64657, Test Acc: 77.63%
Epoch: 50 | Loss: 0.30438, Acc: 89.26% | Test Loss: 0.45135, Test Acc: 85.27%
Epoch: 75 | Loss: 0.16542, Acc: 94.02% | Test Loss: 0.38262, Test Acc: 88.59%
Epoch: 100 | Loss: 0.10426, Acc: 96.69% | Test Loss: 0.38542, Test Acc: 89.78%
Epoch: 125 | Loss: 0.07353, Acc: 97.64% | Test Loss: 0.39617, Test Acc: 90.23%
1. Preprocessing
2.0 Loading the model
Country 10
Results => Country ( 2 )
real_class 2
Epoch: 0 | Loss: 2.28975, Acc: 12.24% | Test Loss: 1.90043, Test Acc: 35.08%
Epoch: 25 | Loss: 0.58844, Acc: 79.47% | Test Loss: 0.70518, Test Acc: 77.01%
Epoch: 50 | Loss: 0.29766, Acc: 89.39% | Test Loss: 0.50104, Test Acc: 84.79%
Epoch: 75 | Loss: 0.16474, Acc: 94.41% | Test Loss: 0.45140, Test Acc: 87.61%
Epoch

/home/payetquentin/Documents/workspace/briefs/S9_music/Music/.venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


2.0 Loading the model
Country 76
Blues 2
Jazz 2
Rock 1
Results => Country ( 2 )
real_class 2
Epoch: 0 | Loss: 2.28943, Acc: 12.87% | Test Loss: 1.89127, Test Acc: 36.97%
Epoch: 25 | Loss: 0.61189, Acc: 78.52% | Test Loss: 0.64222, Test Acc: 77.04%
Epoch: 50 | Loss: 0.31755, Acc: 89.06% | Test Loss: 0.44275, Test Acc: 86.18%
Epoch: 75 | Loss: 0.16848, Acc: 93.93% | Test Loss: 0.37957, Test Acc: 88.84%
Epoch: 100 | Loss: 0.10627, Acc: 96.32% | Test Loss: 0.37033, Test Acc: 90.12%
Epoch: 125 | Loss: 0.07030, Acc: 97.78% | Test Loss: 0.36674, Test Acc: 91.05%
1. Preprocessing


...

2.0 Loading the model
Rock 17
Jazz 2
Blues 1
Pop 1
Classical 18
Country 3
Metal 77
Results => Metal ( 6 )
real_class 6
Epoch: 0 | Loss: 2.28174, Acc: 13.05% | Test Loss: 1.90531, Test Acc: 33.94%
Epoch: 25 | Loss: 0.62280, Acc: 78.58% | Test Loss: 0.68503, Test Acc: 77.51%
Epoch: 50 | Loss: 0.31763, Acc: 89.08% | Test Loss: 0.46880, Test Acc: 85.30%
Epoch: 75 | Loss: 0.18809, Acc: 93.55% | Test Loss: 0.40833, Test Acc: 88.11%
Epoch: 100 | Loss: 0.12046, Acc: 95.74% | Test Loss: 0.40348, Test Acc: 89.60%
Epoch: 125 | Loss: 0.09331, Acc: 96.83% | Test Loss: 0.40812, Test Acc: 90.28%
1. Preprocessing
2.0 Loading the model
Rock 3
Metal 6
Disco 1
Results => Metal ( 6 )
real_class 6
Epoch: 0 | Loss: 2.28178, Acc: 13.29% | Test Loss: 1.90309, Test Acc: 34.71%
Epoch: 25 | Loss: 0.62662, Acc: 78.03% | Test Loss: 0.67287, Test Acc: 78.45%
Epoch: 50 | Loss: 0.32104, Acc: 89.10% | Test Loss: 0.46267, Test Acc: 85.51%
Epoch: 75 | Loss: 0.18317, Acc: 93.79% | Test Loss: 0.40832, Test Acc: 88.08%
Epo

[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


2.0 Loading the model
Rock 1
Jazz 1
Blues 14
Hiphop 4
Pop 62
Classical 3
Reggae 34
Disco 4
Country 7
Results => Pop ( 7 )
real_class 4
Epoch: 0 | Loss: 2.28410, Acc: 13.23% | Test Loss: 1.91079, Test Acc: 32.65%
Epoch: 25 | Loss: 0.64486, Acc: 78.12% | Test Loss: 0.67231, Test Acc: 77.63%
Epoch: 50 | Loss: 0.32461, Acc: 88.49% | Test Loss: 0.44353, Test Acc: 85.61%
Epoch: 75 | Loss: 0.19286, Acc: 93.43% | Test Loss: 0.38317, Test Acc: 88.21%
Epoch: 100 | Loss: 0.13095, Acc: 95.58% | Test Loss: 0.36106, Test Acc: 89.92%
Epoch: 125 | Loss: 0.09549, Acc: 96.72% | Test Loss: 0.35779, Test Acc: 90.35%
1. Preprocessing


/home/payetquentin/Documents/workspace/briefs/S9_music/Music/.venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/home/payetquentin/Documents/workspace/briefs/S9_music/Music/.venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/home/payetquentin/Documents/workspace/briefs/S9_music/Music/.venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/home/payetquentin/Documents/workspace/briefs/S9_music/Music/.venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/home/payetquentin/Documents/workspace/briefs/S9_music/Music/.venv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate t

2.0 Loading the model
Rock 10
Jazz 4
Blues 14
Hiphop 26
Pop 22
Classical 14
Reggae 8
Country 6
Metal 1
Results => Hiphop ( 4 )
real_class 4
Epoch: 0 | Loss: 2.28588, Acc: 12.70% | Test Loss: 1.90708, Test Acc: 32.81%
Epoch: 25 | Loss: 0.64582, Acc: 77.86% | Test Loss: 0.66569, Test Acc: 78.23%
Epoch: 50 | Loss: 0.33971, Acc: 88.09% | Test Loss: 0.44905, Test Acc: 85.42%
Epoch: 75 | Loss: 0.19458, Acc: 93.29% | Test Loss: 0.41129, Test Acc: 88.62%
Epoch: 100 | Loss: 0.12737, Acc: 95.65% | Test Loss: 0.41393, Test Acc: 89.23%
Epoch: 125 | Loss: 0.09446, Acc: 96.73% | Test Loss: 0.41033, Test Acc: 90.12%
1. Preprocessing
2.0 Loading the model
Jazz 56
Blues 1
Pop 4
Classical 2
Disco 2
Country 2
Metal 19
Results => Jazz ( 5 )
real_class 4
Epoch: 0 | Loss: 2.28757, Acc: 12.24% | Test Loss: 1.92277, Test Acc: 33.01%
Epoch: 25 | Loss: 0.64701, Acc: 77.68% | Test Loss: 0.68229, Test Acc: 76.77%
Epoch: 50 | Loss: 0.33020, Acc: 88.89% | Test Loss: 0.45828, Test Acc: 85.02%
Epoch: 75 | Loss: 0.196

# Preprocess data

# Predict

# Grow dataset with new data

# Training